# Lab 9: Missing Values and Cleaning Messy Data

In [1]:
library(tidyverse)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
(stocks <- tibble(
  Year   = c(2015, 2015, 2015, 2015, 2016, 2016, 2016),
  Qtr    = c(   1,    2,    3,    4,    2,    3,    4),
  Return = c(1.88, 0.59, 0.35,   NA, 0.92, 0.17, 2.66)
))

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


## Missing Values
Missing values can be:
1. *Explicit* (marked as NA in our data)
1. *Implicit* (not present in the data)

In this example we have one explicitly missing value for the 4th quarter of 2015. Are there any other missing values? Yes, because we do not have an observation for the first quarter of 2016.

`complete`: Turns implicit missing values into explicit missing values.

In [3]:
stocks %>% complete(Year, Qtr)

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,NA
2016,1,NA
2016,2,0.92
2016,3,0.17
2016,4,2.66


`fill`: Fills missing values in selected columns using the previous entry.

In [4]:
stocks %>% complete(Year, Qtr) %>% fill(Return)

Year,Qtr,Return
<dbl>,<dbl>,<dbl>
2015,1,1.88
2015,2,0.59
2015,3,0.35
2015,4,0.35
2016,1,0.35
2016,2,0.92
2016,3,0.17
2016,4,2.66


The missing values also become `explicit` if we widen the tibble.

In [5]:
stocks_wide = stocks %>% pivot_wider(names_from = Year, values_from = Return)

stocks_wide

Qtr,2015,2016
<dbl>,<dbl>,<dbl>
1,1.88,NA
2,0.59,0.92
3,0.35,0.17
4,NA,2.66


`pivot_longer` will keep all these explicitly missing values by default.

In [6]:
stocks_wide %>% pivot_longer(cols = `2015`:`2016`, names_to = 'Year') %>%
arrange(Year)

Qtr,Year,value
<dbl>,<chr>,<dbl>
1,2015,1.88
2,2015,0.59
3,2015,0.35
4,2015,NA
1,2016,NA
2,2016,0.92
3,2016,0.17
4,2016,2.66


## Cleaning the messy data

In [7]:
datacamp_url = "https://assets.datacamp.com/production/repositories/34/datasets/b3c1036d9a60a9dfe0f99051d2474a54f76055ea/weather.rds"
weather = readRDS(url(datacamp_url))

In [8]:
weather %>% head

,X,year,month,measure,X1,X2,X3,X4,X5,X6,⋯,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,2014,12,Max.TemperatureF,64,42,51,43,42,45,⋯,44,47,46,59,50,52,52,41,30,30
2,2,2014,12,Mean.TemperatureF,52,38,44,37,34,42,⋯,39,45,44,52,44,45,46,36,26,25
3,3,2014,12,Min.TemperatureF,39,33,37,30,26,38,⋯,33,42,41,44,37,38,40,30,22,20
4,4,2014,12,Max.Dew.PointF,46,40,49,24,37,45,⋯,39,45,46,58,31,34,42,26,10,8
5,5,2014,12,MeanDew.PointF,40,27,42,21,25,40,⋯,34,42,44,43,29,31,35,20,4,5
6,6,2014,12,Min.DewpointF,26,17,24,13,12,36,⋯,25,37,41,29,28,29,27,10,-6,1


The first row is for row number, let's ignore that

In [9]:
weather = weather %>% select(-X)

It looks like the values for the weather measurements (column 3) for each day of the month are stored in the columns `X1` to `X31`. From `tidy data` perspective, the data set is messy because:
* column names are values (`X1` to `X31`)
* variable names are represented as values (column 3 - `measure`)

We can correct it by using `pivot_longer`

In [10]:
tidy_weather = weather %>% 
pivot_longer(cols = `X1`:`X31`, names_to = 'day', values_to = "value") %>%
select(year, month, day, everything())

head(tidy_weather)

year,month,day,measure,value
<int>,<int>,<chr>,<chr>,<chr>
2014,12,X1,Max.TemperatureF,64
2014,12,X2,Max.TemperatureF,42
2014,12,X3,Max.TemperatureF,51
2014,12,X4,Max.TemperatureF,43
2014,12,X5,Max.TemperatureF,42
2014,12,X6,Max.TemperatureF,45


The values in the column `measure` of the weather dataset should be variables.

In [11]:
tidy_weather = tidy_weather %>% pivot_wider(names_from = measure, values_from = value)

head(tidy_weather)

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,X1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,X2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,X3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,X4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,X5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,X6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


In [12]:
tidy_weather %>% head

year,month,day,Max.TemperatureF,Mean.TemperatureF,Min.TemperatureF,Max.Dew.PointF,MeanDew.PointF,Min.DewpointF,Max.Humidity,⋯,Max.VisibilityMiles,Mean.VisibilityMiles,Min.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Wind.SpeedMPH,Max.Gust.SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2014,12,X1,64,52,39,46,40,26,74,⋯,10,10,10,22,13,29,0.01,6,Rain,268
2014,12,X2,42,38,33,40,27,17,92,⋯,10,8,2,24,15,29,0.10,7,Rain-Snow,62
2014,12,X3,51,44,37,49,42,24,100,⋯,10,5,1,29,12,38,0.44,8,Rain,254
2014,12,X4,43,37,30,24,21,13,69,⋯,10,10,10,25,12,33,0.00,3,,292
2014,12,X5,42,34,26,37,25,12,85,⋯,10,10,5,22,10,26,0.11,5,Rain,61
2014,12,X6,45,42,38,45,40,36,100,⋯,10,4,0,22,8,25,1.09,8,Rain,313


There are couple of things still odd in this dataset. One of them is name of the days. They start with an `X`. We can fix it by `str_replace` function. In following weeks, we are going to cover it in detail when we cover the strings. Let me quickly remind you the usage:

In [13]:
# Replace $ with nothing, so it removes the dollar sign
cost = c("$8", "12.5$", "$45")
cost = str_replace_all(cost, "\\$", "")
print(cost) 

#change its type to numeric
cost = as.numeric(cost)
print(cost)

[1] "8"    "12.5" "45"  
[1]  8.0 12.5 45.0


#### Exercise 1: Remove `X` from the `day` entries and change its type to `numeric`

#### Exercise 2: Combine the year, month, and day columns into a new column called date. Hint: Use `unite` function

#### Exercise 3: Move events variable to the second column (Just after the `date`)

#### Exercise 4: `PrecipitationIn` has “T”s (Trace). “Traces” in precipitation are defined as less than 0.005 inch. So, in this case, we need to map “T”s to 0.

#### Exercise 5: Other then events, all measures are numeric but coded as chracter. Convert them to numeric

#### Exercise 6: What are the unique events in the dataset?

#### Exercise 7: Empty entry means, there is no event. Change it to `Clear`

Reference: [Cleaning Messy Weather Dataset with tidyverse](https://www.rpubs.com/justinhtet/cleaning-messy-weather-dataset-with-tidyverse)